## Introducción a la Criptografía Asimétrica
### 1. Introducción a RSA
El algoritmo RSA basa su seguridad en la complejidad del problema IFP (factorizacion de numeros enteros). Posee tres etapas: 1. Generación de Claves, 2. Encripción (con clave pública), 3. desencripción (con Clave Privada)
#### Inicializacion

In [18]:
# Imports
import aritmeticaModular as am
import random as random

# El mensaje en claro
print ("EL MENSAJE")
textoPlano='CASA'
print ("mensaje: "+str(textoPlano))
codigoPlano=[]
for i in range(len(textoPlano)):
    num=ord(str(textoPlano[i]))
    codigoPlano.append(num)
print ("codigo mensaje: "+str(codigoPlano))


EL MENSAJE
mensaje: CASA
codigo mensaje: [67, 65, 83, 65]


#### Determinación de Claves Pública y Privada
Implementa RSA con p<umbral, q<umbral 

In [3]:

umbral=999
primos=am.listarPrimos(umbral)
p=primos[random.randint(int(len(primos)/2),len(primos)-1)]
q=primos[random.randint(int(len(primos)/2),len(primos)-1)]
#p=17
#q=19
n=p*q
phi=(p-1)*(q-1)
e=primos[random.randint(int(len(primos)/2),len(primos)-1)]
while(e==p or e==q):
    e=primos[random.randrange(int(len(primos)/2),len(primos))]
#e=47
d=am.inverso(e,phi)
K_pub=(e,n)
K_priv=(d,n)
print("GENERACION DE CLAVES:")
print("K_pub="+str(K_pub)+" - K_priv="+str(K_priv))


GENERACION DE CLAVES:
K_pub=(599, 581129) - K_priv=(442199, 581129)


#### Encripción con clave pública

In [4]:
codigoCifrado=[]
for i in range(len(codigoPlano)):
    cifra=int(am.calcularPotencia(codigoPlano[i],e,n))
    codigoCifrado.append(cifra)
print ("codigo cifrado: "+str(codigoCifrado))

codigo cifrado: [223346, 364860, 311277, 364860]


#### Desencripción con Clave Privada

In [5]:
codigoDescifrado=[]
for i in range(len(codigoCifrado)):
    descifrado=int(am.calcularPotencia(codigoCifrado[i],d,n))
    codigoDescifrado.append(descifrado)
print (str(codigoDescifrado))
textoDescifrado=''
for i in range(len(codigoDescifrado)):
    textoDescifrado=str(textoDescifrado)+chr(codigoDescifrado[i]%128)
print ("codigo Descifrado: "+str(textoDescifrado))

[67, 65, 83, 65]
codigo Descifrado: CASA


#### Hacking RSA
Se dispone de la clave pública y del mensaje cifrado. Buscaremos obtener la clave privada y el texto en claro. Los pasos: 1. Factorizar n (de la clave pública). 2. obtener phi, 3. obtener d=inverso(e,phi). 3. Desencriptar el mensaje

In [6]:
# Factorizar n
factores=am.factorizar(n)
print("Factorizacion de n="+str(n)+": "+str(factores[0]))
p=factores[1]
q=factores[2]
print("p="+str(p)+" - q="+str(q))

Factorizacion de n=581129: 581129=829*701
p=829 - q=701


In [7]:
# Calcular phi
phi=(p-1)*(q-1)
print("phi(n)="+str(phi))

phi(n)=579600


In [8]:
# Calcular d
d=am.inverso(e,phi)
K_privHack=(d,n)
print("K_privHack="+str(K_privHack))
print("d="+str(d))

K_privHack=(442199, 581129)
d=442199


In [9]:
# Desencriptar con clave hackeada
codigoDescifradoHackeado=[]
for i in range(len(codigoCifrado)):
    descifradoHackeado=int(am.calcularPotencia(codigoCifrado[i],d,n))
    codigoDescifradoHackeado.append(descifradoHackeado)
print (str(codigoDescifradoHackeado))
textoDescifradoHackeado=''
for i in range(len(codigoDescifradoHackeado)):
    textoDescifradoHackeado=str(textoDescifradoHackeado)+chr(codigoDescifradoHackeado[i]%128)
print ("codigo Descifrado Hackeado: "+str(textoDescifradoHackeado)) 


[67, 65, 83, 65]
codigo Descifrado Hackeado: CASA


### 2. Sistema de Intercambio de Claves: Diffie Hellman
#### Inicialización: Parámetros públicos (acordados abiertamente)

In [19]:
p = 23  # Un número primo
g = 5   # Una raíz primitiva de p

print(f"Parámetros públicos:")
print(f"Primo (p): {p}")
print(f"Base (g): {g}")

Parámetros públicos:
Primo (p): 23
Base (g): 5


#### Generación de Claves Privadas
Acá "Alice" y "Bob" eligen sus números secretos. Estos nunca se comparten.

In [21]:
a = 19  # Clave privada de Alice
b = 15 # Clave privada de Bob

print(f"Clave privada de Alice (a): {a}")
print(f"Clave privada de Bob (b): {b}")

Clave privada de Alice (a): 19
Clave privada de Bob (b): 15


#### Cálculo y envío de Claves Públicas
Cada uno calcula su clave pública. Alice: $A = g^a \pmod p$, Alice: $B = g^b \pmod p$. Estas claves son las que se envían por internet.

In [22]:
# Alice calcula: A = g^a mod p
A = (g**a) % p

# Bob calcula: B = g^b mod p
B = (g**b) % p

print(f"Alice envía a Bob A = {A}")
print(f"Bob envía a Alice B = {B}")

Alice envía a Bob A = 7
Bob envía a Alice B = 19


#### Cálculo del Secreto Compartido
Ahora viene la "magia" matemática. Cada uno toma la clave pública del otro y la eleva a su propia clave privada. Ambos deberían obtener el mismo resultado.<br>
$K = B^a \pmod p = A^b  \pmod p$


In [23]:
# Alice calcula el secreto: s = B^a mod p
secreto_alice = (B**a) % p

# Bob calcula el secreto: s = A^b mod p
secreto_bob = (A**b) % p

print(f"Secreto calculado por Alice: {secreto_alice}")
print(f"Secreto calculado por Bob: {secreto_bob}")

if secreto_alice == secreto_bob:
    print("\n¡Éxito! Ambos tienen el mismo secreto compartido.")

Secreto calculado por Alice: 14
Secreto calculado por Bob: 14

¡Éxito! Ambos tienen el mismo secreto compartido.
